In [1]:
import pandas as pd
import numpy as np
import chromadb

# Read in the data
negated_card = pd.read_csv('/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/Clustering/Raw Data/Climate/Negated Claims/negated_card_data_score_over_0.7.csv')
negated_card = negated_card.rename(columns={'text': 'Text', 'claim': 'Category'})

negated_epa = pd.read_csv('/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/Clustering/Raw Data/Climate/Negated Claims/negated_epa_data.csv')
negated_epa = negated_epa.rename(columns={'text': 'Text'})

epa_who_data = pd.read_csv(
            "/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/Clustering/Raw Data/Climate/climate_change_epa_who.csv")
epa_who_data = epa_who_data.rename(columns={'text': 'Text'})

card_data = pd.read_csv(
            "/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/Clustering/Raw Data/Climate/card_train_with_score.csv")
card_data = card_data.rename(columns={'text': 'Text'})



chroma_client = chromadb.PersistentClient(
path="/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/Clustering/Clustering/Chroma")
        
embedding_collection = chroma_client.get_or_create_collection(
    name="climate_claims_embeddings_unchanged",
    metadata={"hnsw:space": "cosine"}
)


def get_embedding(text):
    query_original = embedding_collection.get(ids=[text], include=['embeddings', 'documents', 'metadatas'])
    if query_original and len(query_original['embeddings']) > 0:
        return np.array(query_original['embeddings'][0])
    else:
        return None

# Filter out nan in all df where 'Text' is the column name
negated_card = negated_card.dropna(subset=['Text'])
negated_epa = negated_epa.dropna(subset=['Text'])
epa_who_data = epa_who_data.dropna(subset=['Text'])
card_data = card_data.dropna(subset=['Text'])

negated_card['embedding'] = negated_card['Text'].apply(get_embedding)
negated_epa['embedding'] = negated_epa['Text'].apply(get_embedding)
epa_who_data['embedding'] = epa_who_data['Text'].apply(get_embedding)
card_data['embedding'] = card_data['Text'].apply(get_embedding)

# Drop rows where embedding is None
negated_card = negated_card.dropna(subset=['embedding'])
negated_epa = negated_epa.dropna(subset=['embedding'])
epa_who_data = epa_who_data.dropna(subset=['embedding'])
card_data = card_data.dropna(subset=['embedding'])

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def calculate_similarities_and_join_texts(df1, df2):
    results = []
    embeddings2 = np.stack(df2['embedding'].tolist())
    for i, row in tqdm(df1.iterrows()):
        similarities = cosine_similarity([row['embedding']], embeddings2)
        max_sim_index = np.argmax(similarities)
        print(similarities[0][max_sim_index])
        joined_text = df2.iloc[max_sim_index]['Text']
        results.append(joined_text)
    return results

negated_card['original_text'] = calculate_similarities_and_join_texts(negated_card, card_data)
card_data['negated_text'] = calculate_similarities_and_join_texts(card_data, negated_card)
negated_epa['original_text'] = calculate_similarities_and_join_texts(negated_epa, epa_who_data)
epa_who_data['negated_text'] = calculate_similarities_and_join_texts(epa_who_data, negated_epa)

2it [00:00,  4.10it/s]

0.7807372848507287
0.841116358540976


4it [00:00,  5.23it/s]

0.8914375314200766
0.6804415694380952


6it [00:01,  5.93it/s]

...

18it [00:03,  6.27it/s]

0.9113000558113522
0.9617613804726456


19it [00:03,  5.64it/s]

0.9491899325605061



Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10a851bd0>>
Traceback (most recent call last):
  File "/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/venv2/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt



In [23]:
# Print first row negated_card orignal_text and text
print(negated_card['original_text'][1])
print(negated_card['Text'][1])

In fact, it is the arguably the very purpose for which the IPCC was established in the first place..
In fact, it is not the arguably the very purpose for which the IPCC was established in the first place.


In [24]:
# Write dfs to disk
negated_card.to_csv('/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/Clustering/Raw Data/Climate/Cleaned/negated_card', index=False)
negated_epa.to_csv('/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/Clustering/Raw Data/Climate/Cleaned/negated_epa', index=False)
epa_who_data.to_csv('/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/Clustering/Raw Data/Climate/Cleaned/epa', index=False)
card_data.to_csv('/Users/vinayakkannan/Desktop/Projects/FactChecker/FactChecker/Clustering/Raw Data/Climate/Cleaned/card', index=False)